# Setup

In [1]:
from pandas import *
from pathlib import Path
import pickle
import torch.optim as optim
import torch.nn as nn

In [2]:
DATA_DIR = "/Users/bradleybrown/Desktop/Waterloo/Courses/3A/CO255/DeepSimplexPivotFinder/data_generation/data/4_euclid"
SAVE_DIR = "spinningup/data/euc_4/euc_4_s0"

In [3]:
data_dir = Path(DATA_DIR)
data_files = list(data_dir.glob("*"))

In [4]:
fname = data_files[0]

In [5]:
with open(fname, "rb") as file:
    data = pickle.load(file)

Aarr = np.array(data['A'])
barr = np.array(data['b'])
carr = np.array(data['c'])

/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.
/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """
/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [6]:
from spinup.utils.test_policy import *

In [7]:
env,get_action = load_policy_and_env(SAVE_DIR, DATA_DIR)



Loading from spinningup/data/euc_4/euc_4_s0/pyt_save/model.pt.




In [8]:
fname = osp.join(SAVE_DIR, 'pyt_save', 'model.pt')
print('\n\nLoading from %s.\n\n'%fname)

model = torch.load(fname)



Loading from spinningup/data/euc_4/euc_4_s0/pyt_save/model.pt.




In [9]:
def p(T, idxs):
    if type(T) == torch.Tensor: T = np.array(T)
    if len(T.shape) == 1: T = T.reshape(23,38)
    print (DataFrame( [[round(r,2) for j,r in enumerate(row) if j in idxs] for i,row in enumerate(T)] ))

In [10]:
def oh(idx, size=37):
    return torch.eye(size)[idx]

In [11]:
model.pi.logits_net.train()

Sequential(
  (0): Linear(in_features=874, out_features=64, bias=True)
  (1): Tanh()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): Tanh()
  (4): Linear(in_features=64, out_features=37, bias=True)
  (5): Identity()
)

# Testing

In [12]:
NUM_ITERS = 10000
goal = 26
o, r, d, ep_ret, ep_len, n = env.reset(), 0, False, 0, 0, 0

In [13]:
tab = torch.tensor(o[0], dtype=torch.float32)
cands = torch.tensor(o[1], dtype=torch.bool)
y = oh(goal)
torch.where(cands == False)

(tensor([14, 16, 17, 19, 24, 28]),)

In [14]:
tab.requires_grad = True

In [15]:
soft = nn.Softmax()
loss = nn.BCELoss()

In [16]:
before = tab.clone()
optimizer = optim.Adam([tab], lr=1e-2)

In [17]:
for i in range(NUM_ITERS):
    action = soft(model.pi.logits_net(tab))
    
    l = loss(action,y)
    
    l.backward()
    
    optimizer.step()
    optimizer.zero_grad()

/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [18]:
p(before.detach(), idxs={12,26,37})

       0     1      2
0   0.25  0.00   0.75
1   1.00  2.00   5.50
2  -0.25 -0.50   0.12
3  -1.00  2.00   4.50
4  -0.25  0.00   0.75
5  -0.25  0.00   0.25
6  -0.25  0.50   0.12
7  -0.00 -1.00  -0.00
8   0.25  0.00   0.75
9   0.00  0.00   0.00
10  0.00 -0.50   0.12
11  0.00  1.00   0.00
12  0.00  2.00   2.50
13  0.00  0.00   0.00
14  0.00  0.00   0.00
15  0.00  0.00   0.00
16 -0.25  0.00   0.75
17  0.00  0.00   0.00
18  0.25  0.00   0.25
19  0.00  1.00   0.00
20  0.50  0.50   0.12
21 -0.00 -0.00  -0.00
22  2.08  0.45 -26.82


/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [19]:
p(tab.detach(), idxs={12,26,37})

       0     1      2
0   2.36  1.11   2.87
1  -2.20 -2.92   2.46
2   2.76  2.07   2.24
3  -3.26 -0.81   7.30
4   1.53  1.44   1.81
5   0.86 -2.08  -2.43
6  -4.61 -2.46  -2.59
7  -3.12 -2.34   2.94
8   3.87 -2.57   0.84
9  -2.85  3.15  -2.47
10  2.96 -2.89   1.08
11  4.89  2.66   2.02
12  3.93  1.77   6.08
13 -1.53 -2.34  -1.82
14  0.58 -1.83  -2.85
15  3.00  1.80   3.01
16 -2.42  1.98   2.09
17  3.86 -2.27  -2.83
18  1.66  5.69  -2.41
19  4.76  2.50  -1.84
20  2.96  1.31  -1.39
21 -4.04 -0.67  -2.73
22  4.05 -0.87 -27.15


/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [20]:
p(tab.detach()-before.detach(), idxs={12,26,37})

       0     1     2
0   2.11  1.11  2.12
1  -3.20 -4.92 -3.04
2   3.01  2.57  2.12
3  -2.26 -2.81  2.80
4   1.78  1.44  1.06
5   1.11 -2.08 -2.68
6  -4.36 -2.96 -2.71
7  -3.12 -1.34  2.94
8   3.62 -2.57  0.09
9  -2.85  3.15 -2.47
10  2.96 -2.39  0.96
11  4.89  1.66  2.02
12  3.93 -0.23  3.58
13 -1.53 -2.34 -1.82
14  0.58 -1.83 -2.85
15  3.00  1.80  3.01
16 -2.17  1.98  1.34
17  3.86 -2.27 -2.83
18  1.41  5.69 -2.66
19  4.76  1.50 -1.84
20  2.46  0.81 -1.52
21 -4.04 -0.67 -2.73
22  1.97 -1.31 -0.34


/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  
